### Feature engineering phase 1

- Remove unnecessary columns to reduce dataframe size
- Filter train records according to min/max langtitude and longitude in grid info
- Generate grid info and aggregate records accroding to VIN and grid id

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import timeit
%matplotlib inline

# loading training files
ecar_training_data_path = 'data/training/ecar/'
rcar_training_data_path = 'data/training/rcar/'

ecar_output_filepath = 'data/training/ecar/processed/ecar_training.csv'
rcar_output_filepath = 'data/training/rcar/processed/rcar_training.csv'

weather_filepath = 'data/weather.csv'
training_features_filepath = 'data/training.csv'
test_features_filepath = 'data/test.csv'
submission_filepath = 'data/submit_samples.csv'

ecar_filenames = [f for f in os.listdir(ecar_training_data_path)]
rcar_filenames = [f for f in os.listdir(rcar_training_data_path)]

# define drop columns
ecar_drop_columns = ['work_mode','mileage','avg_fuel_consumption','system_mode']
rcar_drop_columns = ['power_mode','mileage','fuel_consumption']
common_drop_columns = ['lat','lon','date_time']

# training features
feature_columns = ['year','month','day','hour','weekday','grid_id',
                   'temperture','rainy','holiday','car_number']

feature_columns_with_speed = ['year','month','day','hour','weekday','grid_id',
                   'temperture','rainy','holiday','speed','car_number']

feature_drop_to_calc_avg_car_number = ['year','month','day','temperture','rainy','holiday']


# loading grid min/max lantitude and longitude
grids = pd.read_csv('data/grid_info.csv')
grid_max_lantitude = grids['latitude_to'].max()
grid_min_lantitude = grids['latitude_from'].min()
grid_max_longitude = grids['longitude_to'].max()
grid_min_longitude = grids['longitude_from'].min()

In [2]:
def round_down_lat_lon(df):
    df['lat'] = df.lat.round(3)
    df['lon'] = df.lon.round(3)
    return df
        
def filter_data_by_lan_lon(df):
    df = df[(df.lat >= grid_min_lantitude ) & (df.lat <= grid_max_lantitude)]
    df = df[(df.lon >= grid_min_longitude ) & (df.lon <= grid_max_longitude)]
    return df

def add_grid_id(df):
    df_with_grid_id = pd.DataFrame(index=[0], columns=['car_id', 'date_time','speed','lat','lon','grid_id'])
    df_with_grid_id = df_with_grid_id.fillna(-1)
    for i in range(len(grids)):
        min_lan, max_lan = grids['latitude_from'][i], grids['latitude_to'][i]
        min_lon, max_lon, grid_id = grids['longitude_from'][i], grids['longitude_to'][i], grids['grid_id'][i]
        df_ = df[(df.lat >= min_lan ) & (df.lat <= max_lan)]
        df_ = df_[(df_.lon >= min_lon ) & (df_.lon <= max_lon)]
        df_['grid_id'] = grid_id
        df_with_grid_id = df_with_grid_id.append(df_)
    return df_with_grid_id

def format_date(date_time):
    return date_time[:-9].replace('-','')

def retrieve_hour(date_time):
    return date_time[-8:-6]

def preprocess_data(dataType='rcar'):
    # init run time
    start = timeit.default_timer()
    # define common variables
    filenames = rcar_filenames
    filepath = rcar_training_data_path
    drop_columns = rcar_drop_columns
    output_filepath = rcar_output_filepath    
    
    if dataType == 'ecar':
        filenames = ecar_filenames
        filepath = ecar_training_data_path
        drop_columns = ecar_drop_columns
        output_filepath = ecar_output_filepath
        
    # if file is already exist return
    if os.path.exists(output_filepath):
        print(output_filepath + " already existed, will return directly")
        return
    
    df_ = pd.DataFrame(index=[0], columns=['car_id', 'date','hour','speed', 'grid_id'])
    df_ = df_.fillna(-1)
    for filename in filenames:
        # if is directory skip
        if os.path.isdir(filepath + filename):
            continue
        # loading data
        df = pd.read_csv(filepath + filename, low_memory=False)
        # drop uncessary columns
        df = df.drop(columns=drop_columns)
        # filter data by lat and lon
        df = filter_data_by_lan_lon(df)
        # format data_time column
        df['hour'] = df['date_time'].apply(lambda x : retrieve_hour(x))
        df['date'] = df['date_time'].apply(lambda x : format_date(x))
        # generate new column grid_id
        df = add_grid_id(df)
        # filter data by grid_id
        df = df[df.grid_id != -1]
        # drop date_time, lat and lon
        df = df.drop(columns=common_drop_columns)
        # group by below column and get average speed for each record
        df = df.groupby(['car_id', 'date','hour','grid_id'], as_index=False).mean()
        # round average speed
        df.speed = df.speed.round(3)
        # appending df
        df_ = df_.append(df)
        # remove grid_id = -1
        df_ = df_[df_.grid_id != -1]
    # print log and total time
    print("Finish process all the files, total cost:[", timeit.default_timer() - start ,"] seconds.")
    # write final file
    df_.to_csv(output_filepath, index=False)

In [15]:
preprocess_data('rcar')

Finish all the file, total cost:[ 344.8129758159416 ]


In [27]:
preprocess_data('ecar')

data/training/ecar/processed/ecar_training.csv already existed, will return directly


### Feature engineering phase 2
- Merge ecar and rcar training data
- Drop column speed and car_id first(these 2 columns maybe used in future but let's try a simple solution first)
- Aggregate training data and calculate total number of cars per day/per hour/per grid
- Merge training data with weather data

In [3]:
def preprocess_weather_data():
    # split date to year, month and day, adding one more column weekday
    df = pd.read_csv(weather_filepath)
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday
    # format date yyyy-mm-dd to yyyymmdd for later join with car's data
    df['date'] = df['date'].astype(str)
    df['date'] = df['date'].apply(lambda x : x.replace('-', ''))
    df['date'] = df['date'].astype(np.int64)
    return df

def preprocess_phase1_data(use_full_data=True, use_avg_car_number=False):    
    # loading files
    ecar = pd.read_csv(ecar_output_filepath)
    rcar = pd.read_csv(rcar_output_filepath)
    weather = preprocess_weather_data()
    full_data = generate_full_data_set()    
    # merge weather with full data
    f_weather = pd.merge(full_data, weather, on=['year','month','day','hour'], how='left')    
    # merge ecar and rcar data
    df = ecar.append(rcar)    
    # drop car id and speed
    df = df.drop(columns=['car_id','speed'])    
    # add column car_number
    df['car_number'] = 1    
    # aggregate data by date,hour and grid id
    df = df.groupby(['date','hour','grid_id'], as_index=False).sum()    
    # remove unecessary row
    df = df[(df.date != 20161230 ) & (df.date != 20170101)]    
    # merge data set
    df = pd.merge(f_weather, df, on=['date','hour','grid_id'], how='left')    
    # remove 23:00 ~ 8:00 data
    df = remove_other_hours_data(df)    
    # remove CNY
    df = remove_chinese_new_year_data(df)    
    # handle N/A car number
    df = handle_na_car_number(df, use_full_data, use_avg_car_number)    
    # write to disk
    df.to_csv(training_features_filepath, columns=feature_columns, index=False)
    
def generate_average_speed_per_grid_weekday_hour():
    ecar = pd.read_csv(ecar_output_filepath)
    rcar = pd.read_csv(rcar_output_filepath)
    df = ecar.append(rcar)
    # remove outlier
    df = df[(df.date != 20161230 ) & (df.date != 20170101)]
    # remove CNY
    df = df[(df.date != 20170127 ) & (df.date != 20170128)]
    df = df[(df.date != 20170129 ) & (df.date != 20170130)]
    df = df[(df.date != 20170131 ) & (df.date != 20170201)]
    df = df[(df.date != 20170202 )]
    # add new column weekday
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
    df['weekday'] = df['date'].dt.weekday
    # drop uncessary columns
    df = df.drop(columns=['date','car_id'])
    # get average speed per grid, per weekday, per hour
    df = df.groupby(['weekday','hour','grid_id'], as_index=False).mean()
    # round speed
    df['speed'] = df.speed.round() 
    df['speed'] = df['speed'].astype(int)
    return df

# train data from 2017-01-02 to 2017-03-12
def generate_full_data_set():
    jan_df = generate_data_by_range(2017,1,2,31,0,23,1,50)
    feb_df = generate_data_by_range(2017,2,1,28,0,23,1,50)
    march_df = generate_data_by_range(2017,3,1,12,0,23,1,50)
    df = jan_df.append(feb_df)
    df = df.append(march_df)
    return df
    
def generate_data_by_range(year, month, start_date, end_date, start_time, end_time, start_gid, end_gid):
    # define arrays
    years = []
    months = []
    days = []
    hours = []
    grid_id = []
    # generate grid id, date and hour according to criteria
    for date in range(start_date, end_date+1):
        for time in range(start_time, end_time+1):
            for gid in range(start_gid, end_gid+1):
                years.append(year)
                months.append(month)
                days.append(date)
                hours.append(time)
                grid_id.append(gid)
    # genreate dataframe            
    df = pd.DataFrame(data={'year':year,'month':month,'day':days, 'hour': hours, 'grid_id':grid_id})
    df['flag'] = 0
    df=df.reindex(columns=['year','month','day','hour','grid_id','flag'])
    return df

def append_avg_speed_to_train_and_test():
    # loading train and test
    train = pd.read_csv(training_features_filepath)
    test = pd.read_csv(test_features_filepath)
    # if already append speed then return
    if 'speed' in train.columns:
        return
    # generate 
    avg_spd = generate_average_speed_per_grid_weekday_hour()
    # merge with train
    train = pd.merge(train, avg_spd, on=['grid_id','weekday','hour'], how='left')
    # merge with test
    test = pd.merge(test, avg_spd, on=['grid_id','weekday','hour'], how='left')
    # generate new train and test data set
    train.to_csv(training_features_filepath, columns=feature_columns_with_speed, index=False)
    test.to_csv(test_features_filepath, columns=feature_columns_with_speed, index=False)
    
def remove_chinese_new_year_data(df):
    df = df[(df.month != 1) | (df.day != 27)]
    df = df[(df.month != 1) | (df.day != 28)]
    df = df[(df.month != 1) | (df.day != 29)]
    df = df[(df.month != 1) | (df.day != 30)]
    df = df[(df.month != 1) | (df.day != 31)]
    df = df[(df.month != 2) | (df.day != 1)]
    df = df[(df.month != 2) | (df.day != 2)]
    return df

def remove_other_hours_data(df):
    df = df[(df.hour != 0)]
    df = df[(df.hour != 1)]
    df = df[(df.hour != 2)]
    df = df[(df.hour != 3)]
    df = df[(df.hour != 4)]
    df = df[(df.hour != 5)]
    df = df[(df.hour != 6)]
    df = df[(df.hour != 7)]
    df = df[(df.hour != 8)]
    df = df[(df.hour != 23)]
    return df

def handle_na_car_number(df, use_full_data, use_avg_car_number):
    if use_full_data:
        df.fillna(0, inplace = True)
        if use_avg_car_number:
            df = update_car_number_to_avg_if_zero(df)
    else:
        df = df.dropna()
    return df

# update car number to average if zero (per grid/per weekday/per hour)
def update_car_number_to_avg_if_zero(df):
    # generate average car number dataframe
    avg_car = df.drop(columns=feature_drop_to_calc_avg_car_number)
    avg_car = avg_car.groupby(['hour', 'weekday','grid_id'], as_index=False).mean()
    avg_car['car_number'] = avg_car.car_number.round()
    avg_car = avg_car.rename(columns={'car_number': 'avg_car_number'})
    # merge with traing dataframe
    df = pd.merge(df, avg_car, on=['grid_id','weekday','hour'], how='left')
    # found car_number = 0 row and set to average car number
    car_number_index = df.car_number == 0
    df.loc[car_number_index,'car_number'] = df.loc[car_number_index, 'avg_car_number']
    # drop avg_car_number
    df = df.drop(columns=['avg_car_number'])
    return df

In [4]:
preprocess_phase1_data(use_full_data=False, use_avg_car_number=False)
train = pd.read_csv(training_features_filepath)
train.head()

,year,month,day,hour,weekday,grid_id,temperture,rainy,holiday,car_number
0,2017,1,2,9,0,1,13,0,1,20.0
1,2017,1,2,9,0,2,13,0,1,7.0
2,2017,1,2,9,0,3,13,0,1,4.0
3,2017,1,2,9,0,4,13,0,1,3.0
4,2017,1,2,9,0,6,13,0,1,9.0


### Generating test features for submission

- from 20170313 to 20170326 2 weeks data
- 9<=hour<=22's data

In [5]:
def generate_sample_submission_file(start_date, end_date, start_time, end_time, start_gid, end_gid):
    # define arrays
    grid_id = []
    dates = []
    hours = []
    # generate grid id, date and hour according to criteria
    for date in range(start_date, end_date+1):
        for time in range(start_time, end_time+1):
            for gid in range(start_gid, end_gid+1):
                grid_id.append(gid)
                dates.append('201703' + str(date))
                hours.append(time)
    # genreate dataframe            
    df = pd.DataFrame(data={'grid_id': grid_id, 'date': dates, 'hour': hours})
    df['car_number'] = 0
    df=df.reindex(columns=['grid_id','date','hour','car_number'])
    # write to csv
    df.to_csv(submission_filepath, index=False) 


def preprocess_test_data():
    test = pd.read_csv(submission_filepath)
    weather = preprocess_weather_data()
    df = pd.merge(test, weather, on=['date', 'hour'], how='left')
    df.to_csv(test_features_filepath, columns=feature_columns, index=False)

In [6]:
generate_sample_submission_file(start_date=13, end_date=26, start_time=9, end_time=22, start_gid=1, end_gid=50)

preprocess_test_data()
test = pd.read_csv(test_features_filepath)
test.head()

,year,month,day,hour,weekday,grid_id,temperture,rainy,holiday,car_number
0,2017,3,13,9,0,1,9,1,0,0
1,2017,3,13,9,0,2,9,1,0,0
2,2017,3,13,9,0,3,9,1,0,0
3,2017,3,13,9,0,4,9,1,0,0
4,2017,3,13,9,0,5,9,1,0,0


### Append average speed to train and test datasets

In [15]:
append_avg_speed_to_train_and_test()
# show trian data after append with avgerage speed
train = pd.read_csv(training_features_filepath)
train.head()

,year,month,day,hour,weekday,grid_id,temperture,rainy,holiday,speed,car_number
0,2017,1,2,9,0,1,13,0,1,21,20.0
1,2017,1,2,9,0,2,13,0,1,16,7.0
2,2017,1,2,9,0,3,13,0,1,22,4.0
3,2017,1,2,9,0,4,13,0,1,72,3.0
4,2017,1,2,9,0,6,13,0,1,41,9.0


In [16]:
# show test data after append with avgerage speed
test = pd.read_csv(test_features_filepath)
test.head()

,year,month,day,hour,weekday,grid_id,temperture,rainy,holiday,speed,car_number
0,2017,3,13,9,0,1,9,1,0,21,0
1,2017,3,13,9,0,2,9,1,0,16,0
2,2017,3,13,9,0,3,9,1,0,22,0
3,2017,3,13,9,0,4,9,1,0,72,0
4,2017,3,13,9,0,5,9,1,0,72,0


In [25]:
from sklearn.cluster import MiniBatchKMeans,KMeans
from sklearn.metrics import calinski_harabaz_score
import time

train_for_clustering = train.drop(columns=['year','month','day','grid_id','car_number'])

for n_clusters in range(3, 201):
    mbk = MiniBatchKMeans(init='k-means++', n_clusters=n_clusters, batch_size=128 ,n_init=10, max_no_improvement=10, verbose=0)
    mbk.fit(train_for_clustering)
    pred = mbk.predict(train_for_clustering)
    score = calinski_harabaz_score(train_for_clustering, pred)
    print("When cluster is {}, score is {}".format(n_clusters, score))


When cluster is 3, score is 69941.19014081002
When cluster is 4, score is 60056.08231337371
When cluster is 5, score is 52836.312728872304
When cluster is 6, score is 46095.53555225386
When cluster is 7, score is 42863.13478967278
When cluster is 8, score is 40605.82019899379
When cluster is 9, score is 38390.79426651683
When cluster is 10, score is 36654.21364336126
When cluster is 11, score is 34424.49653881763
When cluster is 12, score is 33427.47881870069
When cluster is 13, score is 32031.98248384285
When cluster is 14, score is 32114.303971560035
When cluster is 15, score is 30475.719484839923
When cluster is 16, score is 29656.18722364567
When cluster is 17, score is 29673.0180910773
When cluster is 18, score is 27633.084556455735
When cluster is 19, score is 27543.59374401571
When cluster is 20, score is 26349.71657066581
When cluster is 21, score is 26697.789528841637
When cluster is 22, score is 25055.10085613132
When cluster is 23, score is 25546.237473946545
When cluster is

When cluster is 174, score is 9517.017899948867
When cluster is 175, score is 9563.098994727326
When cluster is 176, score is 9384.198037921822
When cluster is 177, score is 9489.3769225814
When cluster is 178, score is 9349.678618051505
When cluster is 179, score is 9394.252542887756
When cluster is 180, score is 9503.063861693532
When cluster is 181, score is 9306.922407675866
When cluster is 182, score is 9408.895809519716
When cluster is 183, score is 9272.745124308936
When cluster is 184, score is 9295.196195107159
When cluster is 185, score is 9324.559670386781
When cluster is 186, score is 9326.742448471014
When cluster is 187, score is 8545.602688116405
When cluster is 188, score is 9263.431689536043
When cluster is 189, score is 9224.44362082947
When cluster is 190, score is 9284.115731778535
When cluster is 191, score is 9142.300435974843
When cluster is 192, score is 9153.427261047122
When cluster is 193, score is 9080.547343243301
When cluster is 194, score is 9096.88444392